<a href="https://colab.research.google.com/github/yoonjong8739/textmining_python/blob/15.BERT_finetuning/15_BERT_%EC%82%AC%EC%A0%84%ED%95%99%EC%8A%B5_%EB%AA%A8%ED%98%95%EC%97%90_%EB%8C%80%ED%95%9C_%EB%AF%B8%EC%84%B8%EC%A1%B0%EC%A0%95%ED%95%99%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 15.1 BERT 학습을 위한 전처리

In [1]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 50.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 32.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 103.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 77.8 MB/s eta 0:00:00


In [2]:
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 5.1 MB/s eta 0:00:00


In [3]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.5 MB/s eta 0:00:00


In [4]:
import os
PATH = './result'
if not os.path.exists(PATH):
    os.mkdir(PATH)

In [5]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')  # 미세조정학습 없이 언어 모델에 대한 사전학습만 수행한 모형

sentence1 = "What a beautiful day!"
sentence2 = "Nvidia Titan XP has 12GB of VRAM"

# 1. 토큰화 결과
print(sentence1, '토큰화 결과:', tokenizer.tokenize(sentence1))
print(sentence2, '토큰화 결과:', tokenizer.tokenize(sentence2))

What a beautiful day! 토큰화 결과: ['what', 'a', 'beautiful', 'day', '!']
Nvidia Titan XP has 12GB of VRAM 토큰화 결과: ['n', '##vid', '##ia', 'titan', 'xp', 'has', '12', '##gb', 'of', 'vr', '##am']


In [6]:
# 2. BERT 모형 입력 생성
inputs = tokenizer([sentence1, sentence2], padding=True)
print('BERT 입력 : {}'.format(inputs))

BERT 입력 : {'input_ids': [[101, 2054, 1037, 3376, 2154, 999, 102, 0, 0, 0, 0, 0, 0], [101, 1050, 17258, 2401, 16537, 26726, 2038, 2260, 18259, 1997, 27830, 3286, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}


- input_ids : 토큰 임베딩 결과
- token_type_ids : 구간 임베딩 결과
- attention_mask : 0(마스킹이 되어 셀프 어텐션에서 제외), 1(셀프 어텐션에 포함)

In [7]:
# 3. 두 문장으로 이뤄진 시퀀스에 대한 BERT 모형 입력 생성
inputs = tokenizer(sentence1, sentence2, padding=True)
print('두 문장 시퀀스에 대한 BERT 입력 : ', inputs)

두 문장 시퀀스에 대한 BERT 입력 :  {'input_ids': [101, 2054, 1037, 3376, 2154, 999, 102, 1050, 17258, 2401, 16537, 26726, 2038, 2260, 18259, 1997, 27830, 3286, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


- input_ids에서 101은 [CLS]에 해당, 102는 [SEP]에 해당

## 15.2 트랜스포머의 트레이너를 이용한 미세조정학습

In [8]:
import nltk
nltk.download('movie_reviews')

from nltk.corpus import movie_reviews
from sklearn.model_selection import train_test_split
import numpy as np

fileids = movie_reviews.fileids()  # movie review data에서 file id를 가져옴

# file id를 통해 raw text file을 가져옴
reviews = [ movie_reviews.raw(fileid) for fileid in fileids ]
categories = [ movie_reviews.categories(fileid)[0] for fileid in fileids ]

# label을 0, 1의 값으로 변환
label_dict = {'pos':1, 'neg':0}
y = [ label_dict[c] for c in categories ]

X_train, X_test, y_train, y_test = train_test_split(reviews, y, test_size=0.2, random_state=7)

print('Train set count : ', len(X_train))
print('Test set count : ', len(X_test))

[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Unzipping corpora/movie_reviews.zip.


Train set count :  1600
Test set count :  400


In [9]:
from transformers import BertTokenizerFast, BertForSequenceClassification

tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')  # 언어 모델로만 사전학습이 되어있는 사전학습 모형
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')

# 문서의 토큰 수가 BERT에서 정한 입력의 크기보다 큰 경우는 잘라내고, 적은 경우에는 채워주기 위해 padding=True, truncation=True
train_input = tokenizer(X_train, truncation=True, padding=True, return_tensors='pt')
test_input = tokenizer(X_test, truncation=True, padding=True, return_tensors='pt')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
# 사용자 정의 데이터셋 만들기
import torch

class OurDataset(torch.utils.data.Dataset):
    def __init__(self, inputs, labels):
        self.inputs = inputs
        self.labels = labels

    #  주어진 인덱스 idx 에 해당하는 샘플을 데이터셋에서 불러오고 반환
    def __getitem__(self, idx):
        item = { key: torch.tensor(val[idx]) for key, val in self.inputs.items() }
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    # 데이터셋의 샘플 개수를 반환
    def __len__(self):
        return len(self.labels)

train_dataset = OurDataset(train_input, y_train)
test_dataset = OurDataset(test_input, y_test)

In [11]:
# Trainer 클래스는 기본적으로 손실(loss) 값만 제공한다. 따라서 정확도를 알고 싶다면 정확도를 계산하는 함수를 먼저 준비해야 한다.
from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

<ipython-input-11-f0ad26242ba9>:4: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy")


In [12]:
from transformers import Trainer, TrainingArguments

# Trainer에서 사용할 하이퍼 파라미터 지정
training_args = TrainingArguments(
    output_dir='./results',         # 모형 예측이나 체크포인트 출력 폴더, 반드시 필요
    num_train_epochs=2,             # 학습 에포크 수
    per_device_train_batch_size=8,  # 학습에 사용할 배치 사이즈
    per_device_eval_batch_size=16   # 평가에 사용할 배치 사이즈
)

trainer = Trainer(
    model=model,                        # 학습할 모형
    args=training_args,                 # 위에서 정의한 학습 매개변수
    train_dataset=train_dataset,        # 학습 데이터셋
    compute_metrics=compute_metrics
)

# 미세조정학습 실행
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
<ipython-input-10-2d119462dcf0>:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = { key: torch.tensor(val[idx]) for key, val in self.inputs.items() }


Step,Training Loss


TrainOutput(global_step=400, training_loss=0.703602294921875, metrics={'train_runtime': 307.5506, 'train_samples_per_second': 10.405, 'train_steps_per_second': 1.301, 'total_flos': 841955377152000.0, 'train_loss': 0.703602294921875, 'epoch': 2.0})

In [13]:
trainer.evaluate(eval_dataset = test_dataset)

<ipython-input-10-2d119462dcf0>:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = { key: torch.tensor(val[idx]) for key, val in self.inputs.items() }


{'eval_loss': 0.6928656697273254,
 'eval_accuracy': 0.5125,
 'eval_runtime': 13.9731,
 'eval_samples_per_second': 28.626,
 'eval_steps_per_second': 1.789,
 'epoch': 2.0}

## 15.3 파이토치를 이용한 미세조정학습

In [14]:
# 메모리 리셋 - 다시 시작 or 커널 리셋
del model
del trainer
torch.cuda.empty_cache()

In [15]:
from torch.utils.data import DataLoader

train_loader = DataLoader(dataset=train_dataset, shuffle=True, batch_size=8)

In [16]:
from transformers import BertModel

#tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')  # 언어 모델로만 사전학습이 되어있는 사전학습 모형
bert_model = BertModel.from_pretrained('bert-base-uncased')

In [17]:
# BERT를 포함한 신경망 모형
class MyModel(torch.nn.Module):
    def __init__(self, pretrained_model, token_size, num_labels):
        super(MyModel, self).__init__()
        self.token_size = token_size  # BERT 모형의 출력 벡터 크기
        self.num_labels = num_labels  # 분류할 클래스 수
        self.pretrained_model = pretrained_model

        # 분류기 정의
        self.classifier = torch.nn.Linear(self.token_size, self.num_labels)

    def forward(self, inputs):
        # BERT 모형에 입력을 넣고 출력을 받음
        outputs = self.pretrained_model(**inputs)

        # 문서 분류에서는 모든 단어들의 출력(임베딩) 벡터를 사용하지 않고 CLS 토큰에 대응되는 출력 벡터만 사용한다.
        # BERT 출력에서 CLS 토큰에 해당하는 부분만 가져옴
        bert_clf_token = outputs.last_hidden_state[:, 0, :]

        return self.classifier(bert_clf_token)

# token_size는 BERT 토큰과 동일, bert_model.config.hidden_size로 알 수 있음
model = MyModel(bert_model, num_labels=2, token_size=bert_model.config.hidden_size)

In [19]:
from transformers import AdamW
import torch.nn.functional as F
import time

# GPU 가속을 사용할 수 있으면 device를 cuda로 설정하고, 아니면 cpu로 설정
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

model.to(device)  # 모형을 GPU로 복사
model.train()     # 학습모드로 전환

# 옵티마이저를 트랜스포머가 제공하는 AdamW로 설정
optim = AdamW(model.parameters(), lr = 5e-5)

# 멀티클래스이므로 크로스 엔트로피를 손실함수로 사용
criterion = torch.nn.CrossEntropyLoss()

start = time.time()  # 시작시간 기록
num_epochs = 4  # 학습 epoch를 4회로 설정
for epoch in range(num_epochs):
    total_epoch_loss = 0  # epoch의 총 loss 초기화

    for step, batch in enumerate(train_loader):
        optim.zero_grad()  # 그레디언트 초기화

        # 배치에서 라벨을 제외한 입력만 추출해 GPU로 복사
        inputs = { k: v.to(device) for k, v in batch.items() if k != 'labels' }
        labels = batch['labels'].to(device)  # 배치에서 라벨을 추출해 GPU로 복사
        outputs = model(inputs)  # 모형으로 결과 예측

        # 두 클래스에 대해 예측하고 각각 비교해야 하므로
        # labels에 대해 원-핫 인코딩을 적용한 후에 손실을 계산
        loss = criterion(outputs, F.one_hot(labels, num_classes=2).float())  # loss 계산

        if (step + 1) % 100 == 0:
            elapsed = time.time() - start
            print('Epoch %d, batch %d, elapsed time : %.2f, loss: %.4f' % (epoch+1, step+1, elapsed, loss))

        total_epoch_loss += loss
        loss.backward()  # 그레이디언트 계산
        optim.step()  # 가중치 업데이트

    avg_epoch_loss = total_epoch_loss / len(train_loader)  # epoch의 평균 loss 계산
    print('Average loss for epoch %d: %.4f' % (epoch+1, avg_epoch_loss))

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
<ipython-input-10-2d119462dcf0>:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = { key: torch.tensor(val[idx]) for key, val in self.inputs.items() }


Epoch 1, batch 100, elapsed time : 77.65, loss: 0.4598
Epoch 1, batch 200, elapsed time : 155.27, loss: 0.2147
Average loss for epoch 1: 0.5310
Epoch 2, batch 100, elapsed time : 232.85, loss: 0.2445
Epoch 2, batch 200, elapsed time : 310.17, loss: 0.7648
Average loss for epoch 2: 0.2727
Epoch 3, batch 100, elapsed time : 387.41, loss: 0.0277
Epoch 3, batch 200, elapsed time : 464.97, loss: 0.0088
Average loss for epoch 3: 0.1345
Epoch 4, batch 100, elapsed time : 543.62, loss: 0.0219
Epoch 4, batch 200, elapsed time : 621.86, loss: 0.0368
Average loss for epoch 4: 0.0576


In [20]:
from datasets import load_metric

test_loader = DataLoader(dataset=test_dataset, shuffle=True, batch_size=16)

metric = load_metric("accuracy")
model.eval()

for batch in test_loader:
    inputs = { k:v.to(device) for k, v in batch.items() if k != 'labels' }
    labels = batch['labels'].to(device)

    with torch.no_grad():  # 학습할 필요가 없으므로 그레이디언트 계산을 끔
        outputs = model(inputs)
        #print(outputs)

    predictions = torch.argmax(outputs, dim=-1)
    metric.add_batch(predictions=predictions, references=labels)

metric.compute()

<ipython-input-10-2d119462dcf0>:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = { key: torch.tensor(val[idx]) for key, val in self.inputs.items() }


{'accuracy': 0.825}